In [ ]:


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Libraries

In [ ]:


import pandas as pd # data processing
import numpy as np # linear algebra
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, roc_curve, precision_score, roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sn

# Data

In [ ]:


train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')



In [ ]:
train.shape


In [ ]:
train.head()

In [ ]:
train.describe()

In [ ]:
train.info

In [ ]:
fig, scatter = plt.subplots(figsize = (16,9))
corrMatrix = train.corr()
scatter = sn.heatmap(corrMatrix, annot = True)
scatter

 # Preprocessing 

In [ ]:
dataset = pd.concat([train, test])

In [ ]:
train_cat_cols = [f"cat{i}" for i in range(10)]
train_num_col = [f"cont{i}" for i in range(14)]

 LabelEncoder 

In [ ]:
for col in train_cat_cols:
    le = LabelEncoder()
    le.fit(dataset[col])
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [ ]:
df_train = train.sample(frac=0.9, random_state=0)
df_valid = train.drop(df_train.index)


X_train = df_train[[col for col in train.columns if col in (train_cat_cols + train_num_col)]]
X_valid = df_valid[[col for col in train.columns if col in (train_cat_cols + train_num_col)]]
y_train = df_train['target']
y_valid = df_valid['target']

# Model


 CatBoostRegressor

In [ ]:
model = CatBoostRegressor(n_estimators= 850, task_type="GPU",  loss_function= 'RMSE', eval_metric='RMSE', metric_period=1000, use_best_model=True,random_seed=42)

In [ ]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

# Submission 

In [ ]:
pred = model.predict(test[(train_cat_cols + train_num_col)])
sub["target"] = pred
sub.to_csv('submission.csv', index=False)

In [ ]:


pd.read_csv('submission.csv')

